# EDA with RBM

In [16]:
from deap import base, tools, creator
from sklearn.neural_network import BernoulliRBM
import tensorflow as tf
import numpy as np
import random, math
!pip install tqdm &> /dev/null
from tqdm.notebook import trange, tqdm
import time
import deap.benchmarks.binary

## REDA implementation

In [17]:
class REDA:
    """
    Estimation of distribution algorithm with RBM distribution modeling

    Parameters
    ----------
    ndim : int
        Dimentionality of each individual
    ngen : int, optional
        Number of generations (default 10)
    nind : int, optional
        Number of individuals each generation (default 100)
    """

    def __init__(self, ndim, ngen = 10, nind = 100, n_h = None):
        """
        Parameters
        ----------
        ndim : int
            Dimentionality of each individual
        ngen : int, optional
            Number of generations (default 10)
        nind : int, optional
            Number of individuals each generation (default 100)
        n_h : int, optional
            Number of hidden units in the RBM hidden layer (default math.log2(ndim))
        """

        self.ndim = ndim
        self.ngen = ngen
        self.nind = nind
        self.n_h = n_h if n_h else int(math.log2(self.ndim))
    
    @staticmethod
    def sel_random(population, k):
        return np.random.choice(len(population), size=k, replace=False)

    @staticmethod
    def sel_best(population, fitness, k, **kwargs):
        return population[np.argsort(fitness)[::-1][:k]]

    @staticmethod
    def sel_tournament(population, fitness, k,  tournsize):
        chosen = []
        for i in range(k):
            aspirants = REDA.sel_random(population, tournsize)
            chosen.append(population[np.argmax(fitness[aspirants], axis=0)])
        return chosen

    @staticmethod
    def sel_roulette(population, fitness, k, **kwargs):
        s_inds = sorted(zip(fitness, population), reverse=True)
        sum_fits = sum(fitness)
        chosen = []
        for i in range(k):
            u = random.random() * sum_fits
            partial_sum = 0
            for fit, ind in s_inds:
                partial_sum += fit
                if partial_sum > u:
                    chosen.append(ind)
                    break

        return chosen

    def eda(self, evaluation_function, selection_function, selection_amount = 0.3, rand_min = 0., rand_max = 1., stoping_criterion_function = None,**kwargs):
        """
        Parameters
        ----------
        evaluation_function: callable
            Funtion to use to evaluate the individuals.
        selection_function: callable, optional
            Function to use to select the best individuals (default tools.selTournament)
        selection_amount: float, optional
            Percentage of the population that will be used to train RBM (default 0.3)
        rand_min : float, optional
            Minimum value for first generated population (default 0.)
        rand_max : float, optional
            Maximum value for first generated population (default 0.)
        stoping_criterion_function: callable, optional
            Function to use to evaluate if some stoping criterion is reached. This function must have two arguments 'individual' and 'fitness' and return a boolean value (default None)
        """
        
        if 'tournsize' in kwargs.keys():
            tournsize = kwargs['tournsize']
        else:
            tournsize = 2
        
        if not callable(evaluation_function):
            raise TypeError("The given Evaluation function is not callable")

        if not callable(selection_function):
            raise TypeError("The given Selection function is not callable")            

        if stoping_criterion_function and not callable(stoping_criterion_function):
            raise TypeError("The given Stoping criterion function is not callable")

        #gen 0
        population = np.array([ [random.randint(0,1) for _ in range(self.ndim)] for _ in range(self.nind) ])
        current_best = None

        for g in trange(self.ngen,position=0, leave=True):
            tqdm.write(f"\n----------Gen {g+1}----------")

            # eval
            fitness = np.array(list(map(evaluation_function,population)))

            #select
            selected = selection_function(population=population, fitness=fitness, k=int(self.nind* selection_amount), tournsize=tournsize)
            current_best = REDA.sel_best(population,fitness,1)

            #checking stoping criterion            
            #TODO check if selected or best individual accomplish stoping criterion


            tqdm.write(f"\tMin population fitness:\t{min(fitness)}")
            tqdm.write(f"\tMax population fitness:\t{max(fitness)}")
            tqdm.write(f"\tMean population fitness:\t{np.mean(fitness)}")
            tqdm.write(f"\tStd population fitness:\t{np.std(fitness)}")
            tqdm.write(f"\tBest population individual:\t{current_best}")
            
            #generate RBM distribution model
            t = time.time()
            rbm = BernoulliRBM(n_components=self.n_h, learning_rate=0.01, n_iter=50)
            rbm.fit(selected)
            t = time.time() - t
            tqdm.write(f"\tBBRBM training elapsed time: {t} seconds")

            #sampling next-gen population with a BBRBM
            t = time.time()
            with tf.Session() as session:
                h_in = np.random.rand(self.nind,self.n_h)
                population_prob = tf.nn.sigmoid(tf.matmul(h_in, rbm.components_) + rbm.intercept_visible_).eval()
                population_prob = population_prob.reshape((self.nind,self.ndim))
                u = np.random.uniform(size=(self.nind,self.ndim))
                population[ u <= population_prob ] = 1
                population[ u > population_prob] = 0
            
            t= time.time() - t
            tqdm.write(f"\tSampling elapsed time: {t} seconds")

        return evaluation_function(current_best),current_best

## Fitness Functions

In [18]:
def onemax(individual):
    return np.sum(individual)
def onemin(individual):
    return np.sum(-individual)

In [19]:
reda = REDA(10,nind=500, ngen = 50)
reda.eda(onemax, REDA.sel_tournament, tournsize=2)


----------Gen 1----------
	Min population fitness:	1
	Max population fitness:	10
	Mean population fitness:	5.052
	Std population fitness:	1.5701261095848322
	Best population individual:	[[1 1 1 1 1 1 1 1 1 1]]
	BBRBM training elapsed time: 0.09911394119262695 seconds
	Sampling elapsed time: 0.08663249015808105 seconds

----------Gen 2----------
	Min population fitness:	1
	Max population fitness:	8
	Mean population fitness:	4.65
	Std population fitness:	1.4323058332632734
	Best population individual:	[[1 0 1 0 1 1 1 1 1 1]]
	BBRBM training elapsed time: 0.12504863739013672 seconds
	Sampling elapsed time: 0.08608722686767578 seconds

----------Gen 3----------
	Min population fitness:	1
	Max population fitness:	9
	Mean population fitness:	5.094
	Std population fitness:	1.2414362649769821
	Best population individual:	[[1 1 1 1 1 1 1 0 1 1]]
	BBRBM training elapsed time: 0.11259698867797852 seconds
	Sampling elapsed time: 0.0843970775604248 seconds

----------Gen 4----------
	Min populatio

(9, array([[1, 1, 1, 1, 1, 1, 0, 1, 1, 1]]))

In [20]:
reda.eda(onemin, REDA.sel_tournament, tournsize=2)


----------Gen 1----------
	Min population fitness:	-10
	Max population fitness:	-1
	Mean population fitness:	-5.104
	Std population fitness:	1.5751774503210743
	Best population individual:	[[1 0 0 0 0 0 0 0 0 0]]
	BBRBM training elapsed time: 0.10382890701293945 seconds
	Sampling elapsed time: 0.1017615795135498 seconds

----------Gen 2----------
	Min population fitness:	-9
	Max population fitness:	-1
	Mean population fitness:	-5.062
	Std population fitness:	1.4177997037663679
	Best population individual:	[[0 0 0 0 0 1 0 0 0 0]]
	BBRBM training elapsed time: 0.12165570259094238 seconds
	Sampling elapsed time: 0.08995819091796875 seconds

----------Gen 3----------
	Min population fitness:	-9
	Max population fitness:	-2
	Mean population fitness:	-5.72
	Std population fitness:	1.0552724766618335
	Best population individual:	[[0 0 0 1 0 0 0 0 1 0]]
	BBRBM training elapsed time: 0.13146042823791504 seconds
	Sampling elapsed time: 0.08654332160949707 seconds

----------Gen 4----------
	Min 

(-2, array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 1]]))